In [11]:
import os
import pandas as pd
import openpyxl

In [88]:
ITALY_PATH = "data\Italy"
INDIA_PATH = "data\India"
excel_path = os.path.join(ITALY_PATH, "Italy.xlsx")
excel_path_india = os.path.join(INDIA_PATH, "India.xlsx")

df_italy = pd.read_excel(excel_path)
df_italy = df_italy[["Number", "Hgb"]]

df_india = pd.read_excel(excel_path_india)
df_india = df_india[["Number", "Hgb"]]

df = pd.concat([df_italy, df_india], ignore_index=True)
df.head()


,Number,Hgb
0,1,9.3
1,2,10.2
2,3,10.7
3,4,11.7
4,5,11.6


In [100]:
import os
import pandas as pd

ITALY_PATH = "data\Italy"

# Normalizar nombres de columnas
df.columns = df.columns.str.strip().str.lower()

# ⚠️ En tu caso las columnas reales son:
# number  | hgb
# así que NO necesitamos renombrar a id

# Convertir hemoglobina a float
df["hgb"] = pd.to_numeric(df["hgb"], errors="coerce")
df = df.dropna(subset=["hgb"])
df["hgb"] = df["hgb"].astype("float32")


# 🔥 Construcción ROBUSTA del image_path (detecta jpg/png automáticamente)
def build_image_path(row):
    folder = str(int(row["number"]))
    folder_path = os.path.join(ITALY_PATH, folder)

    if not os.path.exists(folder_path):
        return None

    files = os.listdir(folder_path)

    image_files = [
        f for f in files
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    if len(image_files) == 0:
        return None

    return os.path.join(folder_path, image_files[0])


df["image_path"] = df.apply(build_image_path, axis=1)

# Eliminar filas sin imagen válida
df = df[df["image_path"].notnull()]
df = df[df["image_path"].apply(os.path.exists)]

# Agregar país
df["country"] = "Italy"

df.info()


<class 'pandas.DataFrame'>
Index: 202 entries, 0 to 217
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   number      202 non-null    int64   
 1   hgb         202 non-null    float32 
 2   image_path  202 non-null    str     
 3   country     202 non-null    str     
 4   hgb_bin     202 non-null    category
dtypes: category(1), float32(1), int64(1), str(2)
memory usage: 7.4 KB


In [90]:
def preprocess(image_path, label):
    # ensure image_path tensor has string dtype
    image_path = tf.convert_to_tensor(image_path, dtype=tf.string)
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


image_paths, labels = load_data(DATASET_PATH)

# convert lists to tensors with correct dtypes before creating the dataset
image_paths = tf.constant(image_paths, dtype=tf.string)
labels = tf.constant(labels, dtype=tf.float32)

In [91]:
import numpy as np

df["hgb_bin"] = pd.qcut(df["hgb"], q=5, duplicates="drop")

train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["hgb_bin"]
)

# quitar columna auxiliar
train_df = train_df.drop(columns=["hgb_bin"])
val_df = val_df.drop(columns=["hgb_bin"])


In [92]:
print(train_df["hgb"].describe())
print(val_df["hgb"].describe())

count    161.000000
mean      12.679502
std        2.387963
min        7.000000
25%       10.700000
50%       13.000000
75%       14.800000
max       17.400000
Name: hgb, dtype: float64
count    41.000000
mean     12.635853
std       2.374454
min       7.800000
25%      11.300000
50%      13.200000
75%      14.500000
max      17.100000
Name: hgb, dtype: float64


In [93]:
import numpy as np

mean_hgb = train_df["hgb"].mean()
std_hgb = train_df["hgb"].std()

train_df["hgb_norm"] = (train_df["hgb"] - mean_hgb) / std_hgb
val_df["hgb_norm"] = (val_df["hgb"] - mean_hgb) / std_hgb

print("Mean:", mean_hgb)
print("Std:", std_hgb)


Mean: 12.6795025
Std: 2.3879633


In [94]:
import tensorflow as tf

IMG_SIZE = 224
BATCH_SIZE = 16

def preprocess(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32)
    
    image = tf.keras.applications.efficientnet.preprocess_input(image)

    label = tf.cast(label, tf.float32)

    return image, label


In [95]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.05),
    tf.keras.layers.RandomZoom(0.1),
])


In [96]:
def make_dataset(df, training=True):
    image_paths = df["image_path"].values
    labels = df["hgb_norm"].values

    ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    ds = ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

    if training:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                    num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.shuffle(500)

    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    return ds


In [102]:
train_ds = make_dataset(train_df, training=True)
val_ds = make_dataset(val_df, training=False)

In [103]:
for images, labels in train_ds.take(1):
    print(images.shape)
    print(labels.shape)


(32, 224, 224, 3)
(32,)


In [104]:
IMG_SIZE = 224

base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base_model.trainable = False  # primero entrenamos solo la cabeza

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)  # salida regresión
])


In [105]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="mse",
    metrics=["mae"]
)


In [106]:
EPOCHS = 10

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 14s 966ms/step - loss: 1.7256 - mae: 1.0747 - val_loss: 0.9506 - val_mae: 0.8124
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 511ms/step - loss: 1.0781 - mae: 0.8327 - val_loss: 0.7995 - val_mae: 0.7343
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 505ms/step - loss: 1.0547 - mae: 0.8210 - val_loss: 0.8098 - val_mae: 0.7550
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 500ms/step - loss: 1.0056 - mae: 0.8242 - val_loss: 0.8212 - val_mae: 0.7578
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 499ms/step - loss: 0.9385 - mae: 0.7544 - val_loss: 0.8252 - val_mae: 0.7400
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 507ms/step - loss: 0.9791 - mae: 0.7682 - val_loss: 0.8446 - val_mae: 0.7495
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 567ms/step - loss: 0.9795 - mae: 0.7847 - val_loss: 0.8708 - val_mae: 0.7729
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 526ms/step - loss: 0.9317 - mae: 0.7694 - val_loss: 0.8652 - val_mae: 0.7576
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 507ms/step - loss: 0.9049 - mae:

In [107]:
model.evaluate(val_ds)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 149ms/step - loss: 0.9016 - mae: 0.7959


[0.9016438722610474, 0.7958959937095642]

In [113]:
print("Length of train_ds:", len(train_ds))

print("Length of val_ds:", len(val_ds))

Length of train_ds: 6
Length of val_ds: 2


In [108]:
print(train_df["hgb"].describe())

count    161.000000
mean      12.679502
std        2.387963
min        7.000000
25%       10.700000
50%       13.000000
75%       14.800000
max       17.400000
Name: hgb, dtype: float64


In [114]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss=tf.keras.losses.Huber(),  # mejor que MSE aquí
    metrics=["mae"]
)


In [115]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        patience=4,
        restore_best_weights=True
    )
]


In [116]:
history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks
)


Epoch 1/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - loss: 0.4294 - mae: 0.8068 - val_loss: 0.4173 - val_mae: 0.7936
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 591ms/step - loss: 0.4277 - mae: 0.8027 - val_loss: 0.4162 - val_mae: 0.7922
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 581ms/step - loss: 0.3589 - mae: 0.7255 - val_loss: 0.4153 - val_mae: 0.7910
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 583ms/step - loss: 0.4326 - mae: 0.8169 - val_loss: 0.4149 - val_mae: 0.7905
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 591ms/step - loss: 0.4155 - mae: 0.7861 - val_loss: 0.4143 - val_mae: 0.7895
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 621ms/step - loss: 0.3899 - mae: 0.7557 - val_loss: 0.4136 - val_mae: 0.7883
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 573ms/step - loss: 0.4208 - mae: 0.7977 - val_loss: 0.4128 - val_mae: 0.7870
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 538ms/step - loss: 0.4105 - mae: 0.7918 - val_loss: 0.4123 - val_mae: 0.7864
Epoch 9/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 538ms/step - loss: 0.4030 - mae: 0.

In [117]:
model.evaluate(val_ds)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - loss: 0.4076 - mae: 0.7804


[0.4075726270675659, 0.7803748846054077]